In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# modifying the following affects the model fit significantly 
batch_size=100
learning_rate = 5e-9
epochs = 500

In [ ]:
input_data = torch.arange(start=0,end=6,step=0.001)
fit_data = torch.sin(input_data)
noise = torch.rand(input_data.shape)-0.5
output_data = fit_data + noise

In [ ]:
plt.subplots(figsize=(2.5,2.5))
plt.plot(input_data, fit_data, 'k:')
plt.plot(input_data[::50], output_data[::50], 'bo', alpha=0.5)

In [ ]:
# lets slice the data into training and test sets

size_data = input_data.shape.numel()
idx_training = 5000
idx_test = idx_training - size_data

perm_indices = torch.randperm(6000)

training_input_data = input_data[perm_indices[:idx_training]]
training_output_data = output_data[perm_indices[:idx_training]]

test_input_data = input_data[~perm_indices[idx_test:]]
test_output_data = output_data[~perm_indices[idx_test:]]

plt.subplots(figsize=(2.5,2.5))
plt.scatter(x=test_input_data[::50], y=test_output_data[::50], color='r', alpha=0.5)
plt.scatter(x=training_input_data[::50], y=training_output_data[::50], color='k', alpha=0.5)

In [ ]:
# datasets and dataloaders
class CustomDataSet(Dataset):
    def __init__(self, input, output):
        self.input = input
        self.output = output

    def __len__(self):
        return len(self.input)

    def __getitem__(self, idx):
        x = self.input[idx]
        y = self.output[idx]
        return x, y

train_dataloader = DataLoader(CustomDataSet(training_input_data, training_output_data), batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(CustomDataSet(test_input_data, test_output_data), batch_size=batch_size, shuffle=True)

In [ ]:
class polynomial_model(nn.Module):
    def __init__(self):
        super().__init__()
        self.a = nn.Parameter(torch.zeros(())) # initially parameters were intitalized with torch.randn(()), the loss function did not converge quickly in that case
        self.b = nn.Parameter(torch.zeros(()))
        self.c = nn.Parameter(torch.zeros(()))
        self.d = nn.Parameter(torch.zeros(()))
        self.e = nn.Parameter(torch.zeros(()))
        self.f = nn.Parameter(torch.zeros(()))
        self.g = nn.Parameter(torch.zeros(()))
    
    def forward(self,x):
        logits = self.a + self.b*x + self.c*(x**2) + self.d*(x**3) + self.e*(x**4) + self.f*(x**5) + self.g*(x**6)
        return logits

model = polynomial_model()
model

In [ ]:
# Loss function
loss_fn = nn.MSELoss()

In [ ]:
# optimizer
polynomial_optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [ ]:
# constructing training and testing
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        pred=model(X)
        loss=loss_fn(pred,y)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        if batch%2==0:
            loss, current = loss.item(), batch*batch_size + len(X)
            print("loss:", loss, current/size)  

def test_loop(dataloader, model, loss_fn):
    model.eval() # set model to evaluation mode
    size =len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0
    with torch.no_grad(): # since in test_mode
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
        test_loss /= num_batches
        print("Test Loss:", test_loss) 

In [ ]:
for t in range(epochs):
    print(f"Epoch {t+1}\n----------------------------------")
    train_loop(train_dataloader, model, loss_fn, polynomial_optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

In [ ]:
list(model.parameters())

In [ ]:
# Evaluating Fit
def polynomial_model(params, X):
    return params[0] + params[1]*X + params[2]*(X**2) + params[3]*(X**3) + params[4]*(X**4) + params[5]*(X**5) + params[6]*(X**6)

In [ ]:
pytorch_fit_data = polynomial_model(list(model.parameters()), input_data).detach().numpy()

In [ ]:
plt.subplots(figsize=(2.5,2.5))
plt.plot(input_data, fit_data, 'b:')
plt.plot(input_data, pytorch_fit_data, 'r:')
plt.plot(input_data[::100], output_data[::100], 'bo', alpha=0.5)